# AWS Services Embeddings Creation

In [ ]:
import json
import os
from typing import Dict, List
from pathlib import Path

In [ ]:
def load_aws_api_documentation(base_path: str, services: List[str]) -> Dict:
    """
    Load AWS API documentation from your JSON files structure.
    
    Args:
        base_path: Path to the directory containing service JSON files
        services: List of service names to load
    
    Returns:
        Dictionary with service -> method -> documentation mapping
    """
    api_docs = {}
    
    for service in services:
        service_file = Path(base_path) / f"{service}.json"
        
        if not service_file.exists():
            print(f"Warning: {service}.json not found, skipping...")
            continue
            
        try:
            # Load service overview
            with open(service_file, 'r') as f:
                service_data = json.load(f)
            
            service_docs = {}
            
            # Get method names from the service file
            method_names = service_data.get('client', {}).get('methods_names', [])
            
            # Load individual method documentation
            for method_name in method_names:
                method_file = Path(base_path) / f"{method_name}.json"
                
                if method_file.exists():
                    try:
                        with open(method_file, 'r') as f:
                            method_data = json.load(f)
                        
                        # Structure the method documentation
                        service_docs[method_name] = {
                            'description': method_data.get('description', ''),
                            'parameters': method_data.get('parameters', []),
                            'return_structure': method_data.get('return_structure', []),
                            'url': method_data.get('url', '')
                        }
                    except Exception as e:
                        print(f"Error loading {method_name}.json: {e}")
                        
            api_docs[service] = service_docs
            print(f"Loaded {service}: {len(service_docs)} methods")
            
        except Exception as e:
            print(f"Error loading {service}.json: {e}")
    
    return api_docs

